In [2]:
#Feature Engineering & Target Creation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [3]:
# Load cleaned data
sales_df = pd.read_csv('data_cleaned/sales_clean.csv')
inventory_df = pd.read_csv('data_cleaned/inventory_clean.csv')  
external_df = pd.read_csv('data_cleaned/external_clean.csv')

In [4]:
# Convert dates
sales_df['date'] = pd.to_datetime(sales_df['date'])
inventory_df['snapshot_date'] = pd.to_datetime(inventory_df['snapshot_date'])
external_df['date'] = pd.to_datetime(external_df['date'])

print("Data loaded successfully!")
print(f"Sales records: {len(sales_df)}")
print(f"Inventory snapshots: {len(inventory_df)}")

Data loaded successfully!
Sales records: 31696
Inventory snapshots: 8343


# CREATE SALES FEATURES

In [5]:
sales_df.head()

,transaction_id,store_id,product_id,date,sales_quantity,unit_price,total_revenue,month,day_of_week,year,is_weekend,name,type,size,city,category,unit_cost
0,TXN72449,ST001,P001,2022-01-01,4,180.76,723.04,1,Saturday,2022,True,Downtown Grocery,grocery,medium,Boston,grocery,107.38
1,TXN25921,ST001,P003,2022-01-01,16,340.00,5440.00,1,Saturday,2022,True,Downtown Grocery,grocery,medium,Boston,grocery,159.61
2,TXN12345,ST001,P004,2022-01-01,14,239.67,3355.38,1,Saturday,2022,True,Downtown Grocery,grocery,medium,Boston,grocery,135.62
3,TXN61515,ST001,P006,2022-01-01,9,309.04,2781.36,1,Saturday,2022,True,Downtown Grocery,grocery,medium,Boston,grocery,151.07
4,TXN98706,ST001,P009,2022-01-01,12,131.46,1577.52,1,Saturday,2022,True,Downtown Grocery,grocery,medium,Boston,grocery,54.70


In [6]:
# Grouping sales by store, product, and date
daily_sales = sales_df.groupby(['store_id', 'product_id', 'date']).agg({
    'sales_quantity': 'sum',
    'total_revenue': 'sum'
}).reset_index()

In [7]:
print(f"Daily sales aggregated: {len(daily_sales)} records")

Daily sales aggregated: 31696 records


In [8]:
# Sorting by date for time-based features
daily_sales = daily_sales.sort_values(['store_id', 'product_id', 'date'])

In [9]:
# Create moving averages
daily_sales['sales_7day_avg'] = daily_sales.groupby(['store_id', 'product_id'])['sales_quantity'].rolling(7, min_periods=1).mean().values
daily_sales['sales_30day_avg'] = daily_sales.groupby(['store_id', 'product_id'])['sales_quantity'].rolling(30, min_periods=1).mean().values


#We are calculating moving averages to smooth out noise and capture the underlying trend in sales over time.

In [10]:
# Creating lag features (previous day sales)
daily_sales['sales_lag_1'] = daily_sales.groupby(['store_id', 'product_id'])['sales_quantity'].shift(1)
daily_sales['sales_lag_7'] = daily_sales.groupby(['store_id', 'product_id'])['sales_quantity'].shift(7)

#we are calculating lag features to give the model direct access to past values so it can learn how past sales influence future sales.

#### In general we are calculating moving averages and lag features to capture trend over time. which will allow the model to learn from the past behaviors.

In [11]:
# Sales trend (is it increasing or decreasing?)
daily_sales['sales_trend'] = daily_sales['sales_7day_avg'] - daily_sales.groupby(['store_id', 'product_id'])['sales_7day_avg'].shift(7)

#Here we are calculating the change in the 7-day moving average compared to the same day last week for each product in each store

# CREATE DATE FEATURES

In [30]:
# Basic date features
daily_sales['day_of_week'] = daily_sales['date'].dt.dayofweek  # 0=Monday
daily_sales['month'] = daily_sales['date'].dt.month
daily_sales['quarter'] = daily_sales['date'].dt.quarter
daily_sales['is_weekend'] = (daily_sales['date'].dt.dayofweek >= 5).astype(int)


#To help the model understand when sales are happening because time affects consumer behavior.


#Yeah we have to think in a way the model would think 🙂. The model is like baby who doesnt know anything and we need to teach him everything

In [31]:
# Holiday features
# Basically listing the holidays

holidays = ['2022-01-01', '2022-07-04', '2022-11-24', '2022-12-25',
           '2023-01-01', '2023-07-04', '2023-11-23', '2023-12-25',
           '2024-01-01', '2024-07-04', '2024-11-28', '2024-12-25']

holiday_dates = pd.to_datetime(holidays)
daily_sales['is_holiday'] = daily_sales['date'].isin(holiday_dates).astype(int)

#The above creates a binary feature:
#1 = the date is a holiday.
#0 = the date is not a holiday.

In [15]:
# Days until next holiday
def days_to_next_holiday(date):
    future_holidays = [h for h in holiday_dates if h > date]
    if future_holidays:
        return (min(future_holidays) - date).days
    return 365

daily_sales['days_to_holiday'] = daily_sales['date'].apply(days_to_next_holiday)

#For each date in your dataset, it calculates the number of days until the next holiday.

#If there's no future holiday left in the list, it returns 365 (acts like a fallback value).

# MERGE WITH EXTERNAL DATA

In [16]:
# Merge with external data
daily_sales = daily_sales.merge(external_df, on='date', how='left')

#Lets merge the daily sales data with external data to provide extra context it couldn’t learn from internal sales data alone

In [17]:
# Create weather impact features
daily_sales['temp_category'] = pd.cut(daily_sales['temperature'], 
                                     bins=[-50, 32, 70, 100], 
                                     labels=['cold', 'mild', 'hot'])

"""
Temperature affects consumer behavior:

Ice cream sells more in hot weather.

Hot beverages sell more in cold.

Foot traffic may increase in mild temperatures.
"""


daily_sales['has_rain'] = (daily_sales['precipitation'] > 0).astype(int)

"""
Creates a binary feature: 1 if it rained that day, 0 if not.
"""

#its like saying .....if it's raining and cold, people probably stay in and hot chocolate sells more.

'\nCreates a binary feature: 1 if it rained that day, 0 if not.\n'

# CREATE INVENTORY FEATURES

In [18]:
# For each sales record, find the closest inventory snapshot
inventory_features = []

for idx, row in daily_sales.iterrows():
    if idx % 1000 == 0:
        print(f"Processing row {idx}/{len(daily_sales)}")
    
    # Find inventory data for this store/product around this date
    inv_data = inventory_df[
        (inventory_df['store_id'] == row['store_id']) & 
        (inventory_df['product_id'] == row['product_id']) &
        (inventory_df['snapshot_date'] <= row['date'])
    ]
    
    if not inv_data.empty:
        # Get the most recent inventory snapshot
        latest_inv = inv_data.loc[inv_data['snapshot_date'].idxmax()]
        
        inventory_features.append({
            'current_stock': latest_inv['current_stock'],
            'reorder_point': latest_inv['reorder_point'],
            'supplier_lead_time': latest_inv['supplier_lead_time'],
            'days_since_snapshot': (row['date'] - latest_inv['snapshot_date']).days
        })
    else:
        inventory_features.append({
            'current_stock': np.nan,
            'reorder_point': np.nan, 
            'supplier_lead_time': np.nan,
            'days_since_snapshot': np.nan
        })


Processing row 0/143889
Processing row 1000/143889
Processing row 2000/143889
Processing row 3000/143889
Processing row 4000/143889
Processing row 5000/143889
Processing row 6000/143889
Processing row 7000/143889
Processing row 8000/143889
Processing row 9000/143889
Processing row 10000/143889
Processing row 11000/143889
Processing row 12000/143889
Processing row 13000/143889
Processing row 14000/143889
Processing row 15000/143889
Processing row 16000/143889
Processing row 17000/143889
Processing row 18000/143889
Processing row 19000/143889
Processing row 20000/143889
Processing row 21000/143889
Processing row 22000/143889
Processing row 23000/143889
Processing row 24000/143889
Processing row 25000/143889
Processing row 26000/143889
Processing row 27000/143889
Processing row 28000/143889
Processing row 29000/143889
Processing row 30000/143889
Processing row 31000/143889
Processing row 32000/143889
Processing row 33000/143889
Processing row 34000/143889
Processing row 35000/143889
Proce

### In the above code we are looping through every row in daily_sales, and for each row:

- Finds the matching inventory data (inventory_df) for the same store and product, where the inventory snapshot is on or before the sale date.

- From those, it picks the most recent snapshot (idxmax() on snapshot_date).

- Extracts key inventory-related values and appends them as a dictionary to inventory_features.

In [19]:
# Add inventory features to main dataframe
inv_features_df = pd.DataFrame(inventory_features)
daily_sales = pd.concat([daily_sales, inv_features_df], axis=1)


#### The above code merges the engineered inventory features into your main dataset, aligning them with each corresponding sales record.

In [20]:
# Fill missing inventory values
daily_sales['current_stock'].fillna(daily_sales['sales_30day_avg'] * 10, inplace=True)
daily_sales['reorder_point'].fillna(daily_sales['sales_7day_avg'] * 3, inplace=True)
daily_sales['supplier_lead_time'].fillna(7, inplace=True)

C:\Users\MUNNA\AppData\Local\Temp\ipykernel_12596\639307798.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  daily_sales['current_stock'].fillna(daily_sales['sales_30day_avg'] * 10, inplace=True)
C:\Users\MUNNA\AppData\Local\Temp\ipykernel_12596\639307798.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

#### The above code ensures your dataset has no nulls in key inventory features by imputing them based on recent sales trends or a fixed fallback.

(also please ignore the warning 😁)

# CREATE TARGET VARIABLE (RISK CATEGORIES)

In [21]:
# Calculate days of stock remaining
daily_sales['days_of_stock'] = daily_sales['current_stock'] / (daily_sales['sales_7day_avg'] + 0.1)  # +0.1 to avoid division by zero

In [22]:
# Calculate predicted demand for next week
daily_sales['predicted_demand_7days'] = daily_sales['sales_7day_avg'] * 7

In [23]:
# Create risk categories based on business logic
def calculate_risk_category(row):
    days_stock = row['days_of_stock']
    current_stock = row['current_stock']
    reorder_point = row['reorder_point']
    sales_trend = row['sales_trend']
    
    # High risk situations
    if days_stock < 3 or current_stock <= reorder_point:
        return 'HIGH_RISK'
    
    # Stockout risk (low stock with increasing demand)
    elif days_stock < 7 and sales_trend > 0:
        return 'STOCKOUT_RISK'
    
    # Overstock risk (too much stock with decreasing demand) 
    elif days_stock > 30 and sales_trend < -2:
        return 'OVERSTOCK_RISK'
    
    # Normal situation
    else:
        return 'LOW_RISK'

daily_sales['risk_category'] = daily_sales.apply(calculate_risk_category, axis=1)


### It creates a strategic feature (risk_category) that flags whether a product-store-date is at risk of stockout, overstock, or operating normally.

Business logic captured:
- HIGH_RISK: Very low stock or stock already below reorder point.

- STOCKOUT_RISK: Low stock and rising demand-danger of running out.

- OVERSTOCK_RISK: High stock and falling demand-risk of excess inventory.

- LOW_RISK: All other cases-stable inventory situation.



In [24]:
# Create numeric risk score (0-100)
risk_mapping = {'LOW_RISK': 25, 'STOCKOUT_RISK': 65, 'OVERSTOCK_RISK': 60, 'HIGH_RISK': 90}
daily_sales['risk_score'] = daily_sales['risk_category'].map(risk_mapping)


# It transforms the categorical risk_category into a quantitative risk_score that reflects the severity of inventory risk.

In [25]:
print("Target variable created!")
print("\nRisk Category Distribution:")
print(daily_sales['risk_category'].value_counts())

Target variable created!

Risk Category Distribution:
risk_category
LOW_RISK          74649
HIGH_RISK         60072
OVERSTOCK_RISK     7053
STOCKOUT_RISK      2115
Name: count, dtype: int64


# FINAL FEATURE SELECTION

In [26]:
# Select features for modeling
feature_columns = [
    # Sales features
    'sales_7day_avg', 'sales_30day_avg', 'sales_lag_1', 'sales_lag_7', 'sales_trend',
    
    # Date features  
    'day_of_week', 'month', 'quarter', 'is_weekend', 'is_holiday', 'days_to_holiday',
    
    # External features
    'temperature', 'precipitation', 'has_rain', 'competitor_promotion', 'local_event',
    
    # Inventory features
    'current_stock', 'reorder_point', 'supplier_lead_time', 'days_of_stock',
    
    # Target
    'risk_category', 'risk_score'
]

In [28]:
# Keep ID columns
id_columns = ['store_id', 'product_id', 'date']


In [32]:
# Create final dataset
final_features = daily_sales[id_columns + feature_columns].copy()

In [33]:
# Handle remaining missing values
final_features = final_features.fillna(0)

In [34]:
print(f"Final dataset shape: {final_features.shape}")
print(f"Features selected: {len(feature_columns)-2}")  # -2 for target variables

Final dataset shape: (143889, 25)
Features selected: 20


# SAVE PROCESSED DATA

In [35]:
# Save the engineered features
final_features.to_csv('data_processed/features_engineered.csv', index=False)


In [37]:
# Save feature list for later use
feature_list = [col for col in feature_columns if col not in ['risk_category', 'risk_score']]
pd.DataFrame({'feature_name': feature_list}).to_csv('data_processed/feature_list.csv', index=False)